#** Machine learning IB-2024** #** Home task 1.** #Regression, KNN, LinearRegression.

In this homework we will build models to predict the price of an apartment in Russia. below is a description of some columns of the data set. date - date of publication price - price in rubles level- floor, on which is the apartment levels - number of floors in the apartment rooms - number of rooms in the apartment. If value -1, then the apartment is considered apartments. area - area of the apartment. kitchen_area - area of the kitchen. geo_lat - Latitude geo_lon - Longitude building_type - building material. 0 - Don't know. 1 - Other. 2 - Panel. 3 - Monolithic. 4 - Brick. 5 - Blocky. 6 - Wooden

Part 0 – Start of work

To start working with the data, we import the libraries that will be needed in this task.

In [ ]:
import math
import pandas as pd
import numpy as np
import matplotlib as plt
import sklearn
import seaborn as sns

Download the folium library to display the data on the map by coordinates.

In [ ]:
!pip install -q folium

We delete our data from the archive.

In [ ]:
!unzip hw.zip

Archive:  hw.zip
  inflating: input_data.csv          


Download the data from the CSV file into the data frame.

In [ ]:
df = pd.read_csv('input_data.csv',delimiter=';')
df.head()

,date,price,level,levels,rooms,area,kitchen_area,geo_lat,geo_lon,building_type,object_type,postal_code,street_id,id_region,house_id
0,2021-01-01,2451300,15,31,1,30.3,0.0,56.780112,60.699355,0,2,620000.0,NaN,66,1632918.0
1,2021-01-01,1450000,5,5,1,33.0,6.0,44.608154,40.138381,0,0,385000.0,NaN,1,NaN
2,2021-01-01,10700000,4,13,3,85.0,12.0,55.540060,37.725112,3,0,142701.0,242543.0,50,681306.0
3,2021-01-01,3100000,3,5,3,82.0,9.0,44.608154,40.138381,0,0,385000.0,NaN,1,NaN
4,2021-01-01,2500000,2,3,1,30.0,9.0,44.738685,37.713668,3,2,353960.0,439378.0,23,1730985.0


Let us see more in detail what type of data is in our data frame.

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11358150 entries, 0 to 11358149
Data columns (total 15 columns):
 #   Column         Dtype  
---  ------         -----  
 0   date           object 
 1   price          int64  
 2   level          int64  
 3   levels         int64  
 4   rooms          int64  
 5   area           float64
 6   kitchen_area   float64
 7   geo_lat        float64
 8   geo_lon        float64
 9   building_type  int64  
 10  object_type    int64  
 11  postal_code    float64
 12  street_id      float64
 13  id_region      int64  
 14  house_id       float64
dtypes: float64(7), int64(7), object(1)
memory usage: 1.3+ GB


Check the gap.

In [ ]:
df.isna().any()

,0
date,False
price,False
level,False
levels,False
rooms,False
area,False
kitchen_area,False
geo_lat,False
geo_lon,False
building_type,False


Show the coordinates of our buildings on the map.

In [ ]:
import folium
from IPython.display import display

map_df = df.loc[:1000]

m = folium.Map(location=[55.751244, 37.618423], zoom_start=10)

# List of points with width and length
lats = map_df['geo_lat'].loc[:1000]
longs = map_df['geo_lon'].loc[:1000]
# Adding points to the map
for point in zip(lats, longs):
    folium.Marker(
        location=[point[0], point[1]]
    ).add_to(m)

display(m)

Part 1: Preparing data for processing by machine learning models.

**0.5 Ball**. The geography of our observations in the data set is extremely large. However, we know that the cost of apartments in Moscow and St. Petersburg is much higher than the average in Russia. Let's make signs that show whether the apartment is 20 kilometers from the centre of Moscow or whether the apartment is 20 kilometers from the centre of St. Petersburg. Create two signs is_Moscow and is_Saint_Peterburg. To find the distance by the coordinates use the function haversine_distance.

We write our function to calculate the Haversine distance formula taken from the sklearn: https://scikit-learn.org/stable/modules/generated/sklearn.metrics.pairwise.haversine_distances.html

In [ ]:
def haversine_distance(lat1, lon1, lat2, lon2):
    D=2*np.arcsin(np.sqrt(np.sin((np.radians(lat1)-(np.radians(lat2)))/2)**2+np.cos(np.radians(lat1))*np.cos(np.radians(lat2))*np.sin((np.radians(lon1)-np.radians(lon2))/2)**2))
    return D

Lat- width Lon- long

In [ ]:
msc_lat=55.751244 # The width of Moscow
msc_lon=37.618423 # Debt of Moscow
spb_lat=59.937500 # Breadth of St. Petersburg
spb_lon=30.308611 # Dolgoa of Snat-Petersburg

comparison of implementation from scratch with relocation from sklearn

In [ ]:
from sklearn.metrics.pairwise import haversine_distances
from math import radians
msc = [55.751244, 37.618423]
spb = [59.937500, 30.308611]
msc_in_radians = [radians(_) for _ in msc]
spb_in_radians = [radians(_) for _ in spb]
result = haversine_distances([msc_in_radians, spb_in_radians])
result * 6371000/1000  # multiply by Earth radius to get kilometers

array([[  0.        , 634.76301369],
       [634.76301369,   0.        ]])

In [ ]:
earth_radius=6371

In [ ]:
haversine_distance(msc_lat, msc_lon, spb_lat, spb_lon)*earth_radius

634.7630136884759

The distance was the same.

Although Google shows a distance of about 700 km between SPb and Msk, this indicates that the method is not ideal.

In [ ]:
df.columns

Index(['date', 'price', 'level', 'levels', 'rooms', 'area', 'kitchen_area',
       'geo_lat', 'geo_lon', 'building_type', 'object_type', 'postal_code',
       'street_id', 'id_region', 'house_id'],
      dtype='object')

In [ ]:
# We calculate the distance from each point to Moscow and St. Petersburg and decide whether it applies to it or not.
df['is_Moscow'] = (df.apply(lambda row: haversine_distance(msc_lat, msc_lon, row['geo_lat'], row['geo_lon'])*earth_radius, axis=1) <= 20).astype(int)
df['is_Saint_Petersburg'] = (df.apply(lambda row: haversine_distance(spb_lat, spb_lon, row['geo_lat'], row['geo_lon'])*earth_radius, axis=1) <= 20).astype(int)

In [ ]:
df.head()

,date,price,level,levels,rooms,area,kitchen_area,geo_lat,geo_lon,building_type,object_type,postal_code,street_id,id_region,house_id,is_Moscow,is_Saint_Petersburg
0,2021-01-01,2451300,15,31,1,30.3,0.0,56.780112,60.699355,0,2,620000.0,NaN,66,1632918.0,0,0
1,2021-01-01,1450000,5,5,1,33.0,6.0,44.608154,40.138381,0,0,385000.0,NaN,1,NaN,0,0
2,2021-01-01,10700000,4,13,3,85.0,12.0,55.540060,37.725112,3,0,142701.0,242543.0,50,681306.0,0,0
3,2021-01-01,3100000,3,5,3,82.0,9.0,44.608154,40.138381,0,0,385000.0,NaN,1,NaN,0,0
4,2021-01-01,2500000,2,3,1,30.0,9.0,44.738685,37.713668,3,2,353960.0,439378.0,23,1730985.0,0,0


In [ ]:
df.is_Moscow.value_counts()

,count
is_Moscow,
0,10321666
1,1036484


In [ ]:
df.is_Saint_Petersburg.value_counts()

,count
is_Saint_Petersburg,
0,10481742
1,876408


In [ ]:
df.columns

Index(['date', 'price', 'level', 'levels', 'rooms', 'area', 'kitchen_area',
       'geo_lat', 'geo_lon', 'building_type', 'object_type', 'postal_code',
       'street_id', 'id_region', 'house_id', 'is_Moscow',
       'is_Saint_Petersburg'],
      dtype='object')

**0.5 Ball**. In our data set there are signs that we can theoretically use, such as postal_code, but we will do this within the framework of homework very-very long. Therefore it is proposed to remove unnecessary signs from the data frame. We will remove geo_lat, geo_lon, object_type, postal_code, street_id, id_region, house_id.

In [ ]:
df=df.drop(['geo_lat', 'geo_lon','object_type', 'postal_code', 'street_id', 'id_region', 'house_id'],axis=1)
df.head()

,date,price,level,levels,rooms,area,kitchen_area,building_type,is_Moscow,is_Saint_Petersburg
0,2021-01-01,2451300,15,31,1,30.3,0.0,0,0,0
1,2021-01-01,1450000,5,5,1,33.0,6.0,0,0,0
2,2021-01-01,10700000,4,13,3,85.0,12.0,3,0,0
3,2021-01-01,3100000,3,5,3,82.0,9.0,0,0,0
4,2021-01-01,2500000,2,3,1,30.0,9.0,3,0,0


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11358150 entries, 0 to 11358149
Data columns (total 10 columns):
 #   Column               Dtype  
---  ------               -----  
 0   date                 object 
 1   price                int64  
 2   level                int64  
 3   levels               int64  
 4   rooms                int64  
 5   area                 float64
 6   kitchen_area         float64
 7   building_type        int64  
 8   is_Moscow            int64  
 9   is_Saint_Petersburg  int64  
dtypes: float64(2), int64(7), object(1)
memory usage: 866.6+ MB


Check out the gap.

In [ ]:
df.isna().any()

,0
date,False
price,False
level,False
levels,False
rooms,False
area,False
kitchen_area,False
building_type,False
is_Moscow,False
is_Saint_Petersburg,False


There is no data gap now.

In [ ]:
df.to_csv('final_data.csv', index=False)

We keep the final data set so that in any case we do not count the distance again.

Translate date into datetime format

In [ ]:
df=pd.read_csv('final_data.csv',  delimiter=',')

In [ ]:
df.head()

,date,price,level,levels,rooms,area,kitchen_area,building_type,is_Moscow,is_Saint_Petersburg
0,2021-01-01,2451300,15,31,1,30.3,0.0,0,0,0
1,2021-01-01,1450000,5,5,1,33.0,6.0,0,0,0
2,2021-01-01,10700000,4,13,3,85.0,12.0,3,0,0
3,2021-01-01,3100000,3,5,3,82.0,9.0,0,0,0
4,2021-01-01,2500000,2,3,1,30.0,9.0,3,0,0


In [ ]:
df.date=pd.to_datetime(df.date)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11358150 entries, 0 to 11358149
Data columns (total 10 columns):
 #   Column               Dtype         
---  ------               -----         
 0   date                 datetime64[ns]
 1   price                int64         
 2   level                int64         
 3   levels               int64         
 4   rooms                int64         
 5   area                 float64       
 6   kitchen_area         float64       
 7   building_type        int64         
 8   is_Moscow            int64         
 9   is_Saint_Petersburg  int64         
dtypes: datetime64[ns](1), float64(2), int64(7)
memory usage: 866.6 MB


In [ ]:
df.columns

Index(['date', 'price', 'level', 'levels', 'rooms', 'area', 'kitchen_area',
       'building_type', 'is_Moscow', 'is_Saint_Petersburg'],
      dtype='object')

What columns are categorical? What numbers are? Categorical: ('is_Moscow', 'is_Saint_Petersburg', 'building_type', 'rooms') Number: (the rest) Let's encode categorical characters using OneHot-Encoding. https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html

In [ ]:
df.building_type.value_counts()

,count
building_type,
0,7535937
4,1439326
2,1230098
3,718991
1,251398
5,159719
6,22681


is_city is already encoded in int, so it remains building_type

In [ ]:
from sklearn.preprocessing import OneHotEncoder

In [ ]:
ohe = OneHotEncoder()

df_building = pd.DataFrame(ohe.fit_transform(df[['building_type']]).toarray())

df_building.columns = [f'building_type_{i}' for i in range(df_building.shape[1])]

df_building.head()

,building_type_0,building_type_1,building_type_2,building_type_3,building_type_4,building_type_5,building_type_6
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [ ]:
ohe = OneHotEncoder()

df_rooms = pd.DataFrame(ohe.fit_transform(df[['rooms']]).toarray())

df_rooms.columns = [f'rooms_{i}' for i in range(df_rooms.shape[1])]
df_rooms.head()

,rooms_0,rooms_1,rooms_2,rooms_3,rooms_4,rooms_5,rooms_6,rooms_7,rooms_8,rooms_9
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
df = pd.concat([df, df_rooms,df_building], axis=1)
df=df.drop(['building_type','rooms'],axis=1)
df.head()

,date,price,level,levels,area,kitchen_area,is_Moscow,is_Saint_Petersburg,rooms_0,rooms_1,...,rooms_7,rooms_8,rooms_9,building_type_0,building_type_1,building_type_2,building_type_3,building_type_4,building_type_5,building_type_6
0,2021-01-01,2451300,15,31,30.3,0.0,0,0,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2021-01-01,1450000,5,5,33.0,6.0,0,0,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2021-01-01,10700000,4,13,85.0,12.0,0,0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,2021-01-01,3100000,3,5,82.0,9.0,0,0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2021-01-01,2500000,2,3,30.0,9.0,0,0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [ ]:
del df_building, df_rooms

**0.5 Ball**. We work with numerical characters: 1. Add two characters to your dataset: the number of days from the day of the first observation (the difference between the dates of the ads). Maybe for the price forecast the floor is not as important as the ratio of the floor to the number of floors in the house, add this character. After adding a new character the date column can be removed. 2. The numerical characters may have different orders. Let us compare the numerical characters using StandartScaller https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html.

Add a sign from a minimum date

In [ ]:
df.date.min(), df.date.max()

(Timestamp('2021-01-01 00:00:00'), Timestamp('2021-12-31 00:00:00'))

In [ ]:
df.date.dt.day

,date
0,1
1,1
2,1
3,1
4,1
...,...
11358145,31
11358146,31
11358147,31
11358148,31


In [ ]:
df['days_from_min_date']=(df.date-df.date.min()).dt.days
df.head()

,date,price,level,levels,area,kitchen_area,is_Moscow,is_Saint_Petersburg,rooms_0,rooms_1,...,rooms_8,rooms_9,building_type_0,building_type_1,building_type_2,building_type_3,building_type_4,building_type_5,building_type_6,days_from_min_date
0,2021-01-01,2451300,15,31,30.3,0.0,0,0,0.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1,2021-01-01,1450000,5,5,33.0,6.0,0,0,0.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2,2021-01-01,10700000,4,13,85.0,12.0,0,0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0
3,2021-01-01,3100000,3,5,82.0,9.0,0,0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4,2021-01-01,2500000,2,3,30.0,9.0,0,0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0


In [ ]:
df['days_from_min_date'].nunique()

365

In [ ]:
df=df.drop(['date'],axis=1)

In [ ]:
df.head()

,price,level,levels,area,kitchen_area,is_Moscow,is_Saint_Petersburg,rooms_0,rooms_1,rooms_2,...,rooms_8,rooms_9,building_type_0,building_type_1,building_type_2,building_type_3,building_type_4,building_type_5,building_type_6,days_from_min_date
0,2451300,15,31,30.3,0.0,0,0,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1,1450000,5,5,33.0,6.0,0,0,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2,10700000,4,13,85.0,12.0,0,0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0
3,3100000,3,5,82.0,9.0,0,0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4,2500000,2,3,30.0,9.0,0,0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11358150 entries, 0 to 11358149
Data columns (total 25 columns):
 #   Column               Dtype  
---  ------               -----  
 0   price                int64  
 1   level                int64  
 2   levels               int64  
 3   area                 float64
 4   kitchen_area         float64
 5   is_Moscow            int64  
 6   is_Saint_Petersburg  int64  
 7   rooms_0              float64
 8   rooms_1              float64
 9   rooms_2              float64
 10  rooms_3              float64
 11  rooms_4              float64
 12  rooms_5              float64
 13  rooms_6              float64
 14  rooms_7              float64
 15  rooms_8              float64
 16  rooms_9              float64
 17  building_type_0      float64
 18  building_type_1      float64
 19  building_type_2      float64
 20  building_type_3      float64
 21  building_type_4      float64
 22  building_type_5      float64
 23  building_type_6      float64
 

Restart and restart the session.

In [ ]:
df.to_csv('dataset.csv', index=False)

In [ ]:
df=pd.read_csv('dataset.csv', delimiter=',')

In [ ]:
df.head()

,price,level,levels,area,kitchen_area,is_Moscow,is_Saint_Petersburg,rooms_0,rooms_1,rooms_2,...,rooms_8,rooms_9,building_type_0,building_type_1,building_type_2,building_type_3,building_type_4,building_type_5,building_type_6,days_from_min_date
0,2451300,15,31,30.3,0.0,0,0,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1,1450000,5,5,33.0,6.0,0,0,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2,10700000,4,13,85.0,12.0,0,0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0
3,3100000,3,5,82.0,9.0,0,0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4,2500000,2,3,30.0,9.0,0,0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0


In [ ]:
levels=df[['level','levels']].to_numpy()
levels.shape

(11358150, 2)

Add to level/levels

In [ ]:
level_del_levels=levels[:, 0]/levels[:,1]
level_del_levels.shape

<ipython-input-6-433b5cbe4248>:1: RuntimeWarning: divide by zero encountered in divide
  level_del_levels=levels[:, 0]/levels[:,1]
<ipython-input-6-433b5cbe4248>:1: RuntimeWarning: invalid value encountered in divide
  level_del_levels=levels[:, 0]/levels[:,1]


(11358150,)

In [ ]:
# Translate levels into data frame and extend with df
df_level_del_levels = pd.DataFrame(level_del_levels, columns=['level_del_levels'])

df = pd.concat([df, df_level_del_levels], axis=1)

In [ ]:
df.head()

,price,level,levels,area,kitchen_area,is_Moscow,is_Saint_Petersburg,rooms_0,rooms_1,rooms_2,...,rooms_9,building_type_0,building_type_1,building_type_2,building_type_3,building_type_4,building_type_5,building_type_6,days_from_min_date,level_del_levels
0,2451300,15,31,30.3,0.0,0,0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.483871
1,1450000,5,5,33.0,6.0,0,0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1.000000
2,10700000,4,13,85.0,12.0,0,0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0,0.307692
3,3100000,3,5,82.0,9.0,0,0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.600000
4,2500000,2,3,30.0,9.0,0,0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0,0.666667


In [ ]:
df=df.drop(['levels','level'],axis=1)

Keep the final data frame

In [ ]:
df.to_csv('final.csv', index=False)

In [ ]:
from sklearn.preprocessing import StandardScaler

We will open the data frame again and remove the inf and nan values because we have divided it into zero and got some very big/small number, namely level/levels.

In [ ]:
df=pd.read_csv('final.csv', delimiter=',')
df.isna().any()

,0
price,False
area,False
kitchen_area,False
is_Moscow,False
is_Saint_Petersburg,False
rooms_0,False
rooms_1,False
rooms_2,False
rooms_3,False
rooms_4,False


In [ ]:
df.isin([np.inf, -np.inf]).any()

,0
price,False
area,False
kitchen_area,False
is_Moscow,False
is_Saint_Petersburg,False
rooms_0,False
rooms_1,False
rooms_2,False
rooms_3,False
rooms_4,False


In [ ]:
# Removal of lines with NA, inf and -inf
df = df.dropna(subset=df.columns, how='any')
df = df[~np.isinf(df).any(axis=1)]
df = df[~np.isneginf(df).any(axis=1)]

# Verification of the remaining NaN, inf and -inf
print(df.isna().any())
print(df.isin([np.inf, -np.inf]).any())

price                  False
area                   False
kitchen_area           False
is_Moscow              False
is_Saint_Petersburg    False
rooms_0                False
rooms_1                False
rooms_2                False
rooms_3                False
rooms_4                False
rooms_5                False
rooms_6                False
rooms_7                False
rooms_8                False
rooms_9                False
building_type_0        False
building_type_1        False
building_type_2        False
building_type_3        False
building_type_4        False
building_type_5        False
building_type_6        False
days_from_min_date     False
level_del_levels       False
dtype: bool
price                  False
area                   False
kitchen_area           False
is_Moscow              False
is_Saint_Petersburg    False
rooms_0                False
rooms_1                False
rooms_2                False
rooms_3                False
rooms_4                False
ro

In [ ]:
y=df['price'].to_numpy()
X=df.drop(['price'],axis=1).to_numpy()

In [ ]:
del df

In [ ]:
X.shape

(11355998, 23)

Standardization

In [ ]:
std=StandardScaler()
X = std.fit_transform(X)
X.shape

(11355998, 23)

**2 Ball**. Realize the KNNRegressor class, which should make regression by the method to the nearest neighbors.

In [ ]:
from sklearn.neighbors import KDTree
from concurrent.futures import ProcessPoolExecutor

class KNNRegressor:
    def __init__(self, n_neighbors=5, metric='euclidean'):

        self.n_neighbors = n_neighbors
        self.metric = metric
        self.tree = None

    def fit(self, X_train, y_train):
        self.X_train, self.y_train = X_train, y_train
        if self.metric == 'euclidean':
            self.tree = KDTree(X_train)  # Creating a copy of KDTree for training data

    def predict(self, X_test):
        with ProcessPoolExecutor() as executor:
            # Use ProcessPoolExecutor for parallel predictions calculations
            preds = list(executor.map(self.make_preds_single, X_test))
        return np.array(preds)

    def make_preds_single(self, x_test_i):
        # Search for the nearest neighbors for a test example
        distances, indices = self.tree.query(x_test_i.reshape(1, -1), k=self.n_neighbors)
        # Obtaining target values for the nearest neighbors found
        targets = self.y_train[indices[0]]
        # Calculation of the average target variable of the nearest neighbors
        return np.mean(targets)

**3 Balls**. Realize the LinearRegression class that supports gradient leaks SGD, Momentum, AdaGrad. Use gradient to optimize the MSE loss function.

In [ ]:
class LinearRegression:
    def __init__(self, learning_rate=0.01, max_iter=1000, tol=1e-4, optimizer='sgd', epsilon=1e-8, decay_rate=0.9):
        self.learning_rate = learning_rate
        self.max_iter = max_iter
        self.tol = tol
        self.optimizer = optimizer
        self.epsilon = epsilon
        self.decay_rate = decay_rate
        self.w = None
        self.b = None

    def fit(self, X, y):
        n_samples, n_features = X.shape

        # Initialization of Weights and Shifts
        self.w = np.zeros(n_features)
        self.b = 0

        # The choice of optimizer
        if self.optimizer == 'sgd':
            self.optimize_sgd(X, y)
        elif self.optimizer == 'momentum':
            self.optimize_momentum(X, y)
        elif self.optimizer == 'adagrad':
            self.optimize_adagrad(X, y)
        else:
            raise ValueError("Invalid optimizer. Choose 'sgd', 'momentum', or 'adagrad'.")

    def optimize_sgd(self, X, y):
        n_samples, n_features = X.shape
        for i in range(self.max_iter):
            # Calculation of the gradient
            y_pred = np.dot(X, self.w) + self.b
            dw = (1 / n_samples) * np.dot(X.T, (y_pred - y))
            db = (1 / n_samples) * np.sum(y_pred - y)

            # Update of Weight and Shift
            self.w -= self.learning_rate * dw
            self.b -= self.learning_rate * db

            # Verification of compatibility
            if np.linalg.norm(dw) < self.tol and np.abs(db) < self.tol:
                break

    def optimize_momentum(self, X, y):
        n_samples, n_features = X.shape
        v_w = np.zeros(n_features)
        v_b = 0

        for i in range(self.max_iter):
            # Calculation of the gradient
            y_pred = np.dot(X, self.w) + self.b
            dw = (1 / n_samples) * np.dot(X.T, (y_pred - y))
            db = (1 / n_samples) * np.sum(y_pred - y)

            # Update of Weight and Shift using the Moment
            v_w = self.decay_rate * v_w - self.learning_rate * dw
            v_b = self.decay_rate * v_b - self.learning_rate * db
            self.w += v_w
            self.b += v_b

            # Verification of compatibility
            if np.linalg.norm(dw) < self.tol and np.abs(db) < self.tol:
                break

    def optimize_adagrad(self, X, y):
        n_samples, n_features = X.shape
        cache_w = np.zeros(n_features)
        cache_b = 0

        for i in range(self.max_iter):
            # Calculation of the gradient
            y_pred = np.dot(X, self.w) + self.b
            dw = (1 / n_samples) * np.dot(X.T, (y_pred - y))
            db = (1 / n_samples) * np.sum(y_pred - y)

            # Weight updating and shift using AdaGrad
            cache_w += dw ** 2
            cache_b += db ** 2
            self.w -= self.learning_rate / np.sqrt(cache_w + self.epsilon) * dw
            self.b -= self.learning_rate / np.sqrt(cache_b + self.epsilon) * db

            # Verification of compatibility
            if np.linalg.norm(dw) < self.tol and np.abs(db) < self.tol:
                break

    def predict(self, X):
        return np.dot(X, self.w) + self.b

Part 2: Experiments with machine learning models.

**3 Balls**. Do experiments with the machine learning methods you have written. Separate the training and test samples at 0.8 and 0.2 respectively. Measure the MSE, MAE, RMSE error. Use the KNNRegressor and LinearRegression methods from the sklearn library, compare the quality of your solutions and library.

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, root_mean_squared_error, mean_absolute_error

In [ ]:
X.shape, y.shape

((11355998, 23), (11355998,))

In [ ]:
X_train, X_test, y_train, y_test=train_test_split(X,y,test_size=0.2, random_state=42)

**KNN** from Scratch

In [ ]:
knn_regressor = KNNRegressor(n_neighbors=5, metric='euclidean')
knn_regressor.fit(X_train, y_train)

In [ ]:
y_pred = knn_regressor.predict(X_test)

In [ ]:
knn_reg_mse = mean_squared_error(y_test, y_pred)
knn_reg_rmse = root_mean_squared_error(y_test, y_pred)
knn_reg_mae = mean_absolute_error(y_test, y_pred)

print(f'KNN regressor MSE score: {knn_reg_mse}')
print(f'KNN regressor RMSE score: {knn_reg_rmse}')
print(f'KNN regressor MAE score: {knn_reg_mae}')

KNN regressor MSE score: 2.019670104225382e+19
KNN regressor RMSE score: 4494073991.630959
KNN regressor MAE score: 33992143.17584


*KNN* Sklarn

In [ ]:
from sklearn.neighbors import KNeighborsRegressor

In [ ]:
neigh = KNeighborsRegressor(n_neighbors=5, n_jobs=-1,metric='euclidean',algorithm='kd_tree')
neigh.fit(X_train, y_train)

KNeighborsRegressor(algorithm='kd_tree', metric='euclidean', n_jobs=-1)

In [ ]:
preds=neigh.predict(X_test)

In [ ]:
knn_reg_mse = mean_squared_error(y_test, preds)
knn_reg_rmse = root_mean_squared_error(y_test, preds)
knn_reg_mae = mean_absolute_error(y_test, preds)

print(f'MSE score: {knn_reg_mse}')
print(f'RMSE score: {knn_reg_rmse}')
print(f'MAE score: {knn_reg_mae}')

MSE score: 2.019670104210707e+19
RMSE score: 4494073991.614632
MAE score: 33992006.5899


**Linear Regression** from scratch

SGD

In [ ]:
model = LinearRegression(learning_rate=0.01, max_iter=100, tol=1e-4, optimizer='sgd', epsilon=1e-8, decay_rate=0.9)

In [ ]:
model.fit(X_train, y_train)

In [ ]:
preds = model.predict(X_test)

In [ ]:
reg_mse = mean_squared_error(y_test, preds)
reg_rmse = root_mean_squared_error(y_test, preds)
reg_mae = mean_absolute_error(y_test, preds)

print(f'MSE score: {reg_mse}')
print(f'RMSE score: {reg_rmse}')
print(f'MAE score: {reg_mae}')

MSE score: 1.8641296833889904e+17
RMSE score: 431755681.30471545
MAE score: 4175567.594382115


Momentum

In [ ]:
model = LinearRegression(learning_rate=0.01, max_iter=100, tol=1e-4, optimizer='momentum', epsilon=1e-8, decay_rate=0.9)

In [ ]:
model.fit(X_train, y_train)

In [ ]:
preds = model.predict(X_test)

In [ ]:
reg_mse = mean_squared_error(y_test, preds)
reg_rmse = root_mean_squared_error(y_test, preds)
reg_mae = mean_absolute_error(y_test, preds)

print(f'MSE score: {reg_mse}')
print(f'RMSE score: {reg_rmse}')
print(f'MAE score: {reg_mae}')

MSE score: 1.8638640287109357e+17
RMSE score: 431724915.7404441
MAE score: 4201402.059595793


ADAgrad

In [ ]:
model = LinearRegression(learning_rate=0.01, max_iter=100, tol=1e-4, optimizer='adagrad', epsilon=1e-8, decay_rate=0.9)

In [ ]:
model.fit(X_train, y_train)

In [ ]:
preds = model.predict(X_test)

In [ ]:
reg_mse = mean_squared_error(y_test, preds)
reg_rmse = root_mean_squared_error(y_test, preds)
reg_mae = mean_absolute_error(y_test, preds)

print(f'MSE score: {reg_mse}')
print(f'RMSE score: {reg_rmse}')
print(f'MAE score: {reg_mae}')

MSE score: 1.865210614575031e+17
RMSE score: 431880841.7347349
MAE score: 7023666.077774068


Sklearn Lin_reg

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
lin_reg = LinearRegression().fit(X_train, y_train)

In [ ]:
preds=lin_reg.predict(X_test)

In [ ]:
reg_mse = mean_squared_error(y_test, preds)
reg_rmse = root_mean_squared_error(y_test, preds)
reg_mae = mean_absolute_error(y_test, preds)

print(f'MSE score: {reg_mse}')
print(f'RMSE score: {reg_rmse}')
print(f'MAE score: {reg_mae}')

MSE score: 1.863859193452282e+17
RMSE score: 431724355.7470764
MAE score: 4250850.888491383


* Summary *

As a result of the work, the algorithms Haversine distance, KNN and Linear Regression were implemented. (SGD, Momentum, Adagrad), the algorithms were compared to the algorithms implemented in the sklearn by metric. The result was an effective implementation from zero that does not go beyond the finished solution It is worth noting that the method of KNN regression is a costly and low-efficient algorithm in the context of large datasets which is current.